### Task_9

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Cumulative Score").getOrCreate()
df = spark.read.option("header", True).csv("/FileStore/tables/ipl_sample_deliveries.csv")
df.head()

Out[11]: Row(match_id='335982', inning='1', batting_team='Kolkata Knight Riders', bowling_team='Royal Challengers Bangalore', over='0', ball='1', batter='SC Ganguly', bowler='P Kumar', non_striker='BB McCullum', batsman_runs='0', extra_runs='1', total_runs='1', extras_type='legbyes', is_wicket='0', player_dismissed=None, dismissal_kind=None, fielder=None)

In [0]:
window_spec = Window.partitionBy("match_id", "inning") \
                    .orderBy(col("over").cast("int"), col("ball").cast("int")) \
                    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_with_cumulative = df.withColumn("cumulative_score",_sum(col("total_runs").cast("int")).over(window_spec))
df_with_cumulative.select("match_id", "inning", "over", "ball", "total_runs", "cumulative_score").show(20, truncate=False)

+--------+------+----+----+----------+----------------+
|match_id|inning|over|ball|total_runs|cumulative_score|
+--------+------+----+----+----------+----------------+
|335982  |1     |0   |1   |1         |1               |
|335982  |1     |0   |2   |0         |1               |
|335982  |1     |0   |3   |1         |2               |
|335982  |1     |0   |4   |0         |2               |
|335982  |1     |0   |5   |0         |2               |
|335982  |1     |0   |6   |1         |3               |
|335982  |1     |1   |1   |2         |5               |
|335982  |1     |1   |2   |0         |5               |
|335982  |1     |1   |3   |4         |9               |
|335982  |1     |1   |4   |6         |15              |
|335982  |1     |1   |5   |1         |16              |
|335982  |1     |1   |6   |3         |19              |
|335982  |1     |2   |1   |0         |19              |
|335982  |1     |2   |2   |1         |20              |
|335982  |1     |2   |3   |1         |21        